_한국어로 기계번역됨_


# 에이전트 간의 핸드오프 구현 방법


!!! 정보 "전제 조건"
    이 가이드는 다음에 대한 이해를 가정합니다:

    - [다중 에이전트 시스템](../../concepts/multi_agent)
    - [명령](../../concepts/low_level/#command)
    - [LangGraph 용어 집](../../concepts/low_level/)
    

다중 에이전트 아키텍처에서 에이전트는 그래프 노드로 표현될 수 있습니다. 각 에이전트 노드는 단계별로 실행하고 실행을 종료할지 또는 다른 에이전트로 라우팅할지를 결정하며, 잠재적으로 자신으로 라우팅할 수도 있습니다(예: 루프에서 실행). 다중 에이전트 상호작용의 자연스러운 패턴은 [핸드오프](../../concepts/multi_agent#handoffs)로, 한 에이전트가 제어를 다른 에이전트에게 넘기는 것입니다. 핸드오프를 통해 다음을 지정할 수 있습니다:

- **대상**: 탐색할 목표 에이전트 - LangGraph의 노드 이름
- **페이로드**: 해당 에이전트에게 전달할 정보 - LangGraph의 상태 업데이트

LangGraph에서 핸드오프를 구현하기 위해 에이전트 노드는 `Command` 객체를 반환할 수 있습니다. 이 객체를 통해 [제어 흐름과 상태 업데이트를 결합할 수 있습니다](../command):

```python
def agent(state) -> Command[Literal["agent", "another_agent"]]:
    # 라우팅/중지 조건은 무엇이든 될 수 있습니다. 예: LLM 도구 호출 / 구조화된 출력 등.
    goto = get_next_agent(...)  # 'agent' / 'another_agent'
    return Command(
        # 다음에 호출할 에이전트를 지정합니다.
        goto=goto,
        # 그래프 상태 업데이트
        update={"my_state_key": "my_state_value"}
    )
```

가장 일반적인 에이전트 유형 중 하나는 도구 호출 에이전트입니다. 이러한 유형의 에이전트에 대한 한 가지 패턴은 도구 호출에 핸드오프를 감싸는 것입니다. 예를 들면 다음과 같습니다:

```python
@tool
def transfer_to_bob(state):
    """밥에게 전송합니다."""
    return Command(
        goto="bob",
        update={"my_state_key": "my_state_value"},
        # 각 도구 호출 에이전트는 하위 그래프로 구현됩니다.
        # 따라서 다른 에이전트(형제 하위 그래프)로 이동하기 위해
        # 부모 그래프와 관련된 탐색을 지정해야 합니다.
        graph=Command.PARENT,
    )
```

이 가이드는 다음을 보여줍니다:

- `Command`를 사용하여 핸드오프를 구현합니다: 에이전트 노드는 누가 핸드오프할지를 결정하고, `Command`를 통해 핸드오프를 명시적으로 반환합니다. 이는 에이전트가 다른 에이전트로 라우팅되는 방식을 세밀하게 제어해야 할 때 유용합니다. 이는 감독 아키텍처에서 감독 에이전트를 구현하는 데 적합할 수 있습니다.
- 도구를 사용하여 핸드오프를 구현합니다: 도구 호출 에이전트는 `Command`를 통해 핸드오프를 반환할 수 있는 도구에 접근할 수 있습니다. 에이전트 내의 도구 실행 노드는 도구에서 반환된 `Command` 객체를 인식하고 그에 따라 라우팅합니다. 핸드오프 도구는 도구 호출 에이전트를 포함하는 모든 다중 에이전트 시스템에서 유용한 범용 기본 요소입니다.


## 설정


In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-anthropic


In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 파악하고 성능을 개선하세요. LangSmith는 LangGraph로 구축된 LLM 앱을 디버깅, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있게 해줍니다 — 시작하는 방법에 대해 자세히 읽어보세요 <a href="https://docs.smith.langchain.com">여기</a>.
    </p>
</div>


## `Command`를 이용한 핸드오프 구현


두 개의 에이전트로 시스템을 구현해 보겠습니다:

- 덧셈 전문가 (숫자만 더할 수 있음)
- 곱셈 전문가 (숫자만 곱할 수 있음)

이 예제에서 에이전트들은 수학 작업을 수행하기 위해 LLM에 의존할 것입니다. 더 현실적인 [후속 예제](#using-with-a-custom-agent)에서는 수학을 수행하기 위한 도구를 에이전트들에게 제공할 것입니다.

덧셈 전문가가 곱셈이 필요할 때, 곱셈 전문가에게 일을 넘기고 그 반대도 마찬가지입니다. 이는 간단한 다중 에이전트 네트워크의 예입니다.

각 에이전트는 조건부로 `Command` 객체 (예: 핸드오프)를 반환할 수 있는 해당 노드 함수를 가질 것입니다. 노드 함수는 시스템 프롬프트와 다른 에이전트에게 넘겨야 할 때 신호를 보낼 수 있는 도구를 사용하여 LLM을 이용할 것입니다. LLM이 도구 호출로 응답하면, 우리는 `Command(goto=<other_agent>)`를 반환할 것입니다.

> **참고**: LLM이 핸드오프가 필요하다는 신호를 보내기 위해 도구를 사용하고 있지만, 핸드오프 조건은 LLM의 특정 응답 텍스트, LLM의 구조화된 출력, 기타 사용자 정의 논리 등 무엇이든 될 수 있습니다.


In [3]:
from typing_extensions import Literal
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.types import Command

model = ChatAnthropic(model="claude-3-5-sonnet-latest")


@tool
def transfer_to_multiplication_expert():
    """Ask multiplication agent for help."""
    # This tool is not returning anything: we're just using it
    # as a way for LLM to signal that it needs to hand off to another agent
    # (See the paragraph above)
    return


@tool
def transfer_to_addition_expert():
    """Ask addition agent for help."""
    return


def addition_expert(
    state: MessagesState,
) -> Command[Literal["multiplication_expert", "__end__"]]:
    system_prompt = (
        "You are an addition expert, you can ask the multiplication expert for help with multiplication. "
        "Always do your portion of calculation before the handoff."
    )
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    ai_msg = model.bind_tools([transfer_to_multiplication_expert]).invoke(messages)
    # If there are tool calls, the LLM needs to hand off to another agent
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        # NOTE: it's important to insert a tool message here because LLM providers are expecting
        # all AI messages to be followed by a corresponding tool result message
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        return Command(
            goto="multiplication_expert", update={"messages": [ai_msg, tool_msg]}
        )

    # If the expert has an answer, return it directly to the user
    return {"messages": [ai_msg]}


def multiplication_expert(
    state: MessagesState,
) -> Command[Literal["addition_expert", "__end__"]]:
    system_prompt = (
        "You are a multiplication expert, you can ask an addition expert for help with addition. "
        "Always do your portion of calculation before the handoff."
    )
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    ai_msg = model.bind_tools([transfer_to_addition_expert]).invoke(messages)
    if len(ai_msg.tool_calls) > 0:
        tool_call_id = ai_msg.tool_calls[-1]["id"]
        tool_msg = {
            "role": "tool",
            "content": "Successfully transferred",
            "tool_call_id": tool_call_id,
        }
        return Command(goto="addition_expert", update={"messages": [ai_msg, tool_msg]})

    return {"messages": [ai_msg]}


이 두 노드를 하나의 그래프로 결합해 봅시다. 에이전트들 간에는 엣지가 없다는 점에 유의하세요! 만약 전문자가 답을 알고 있다면 사용자에게 직접 전송할 것이고, 그렇지 않으면 다른 전문가에게 도움을 요청할 것입니다.


In [4]:
builder = StateGraph(MessagesState)
builder.add_node("addition_expert", addition_expert)
builder.add_node("multiplication_expert", multiplication_expert)
# we'll always start with the addition expert
builder.add_edge(START, "addition_expert")
graph = builder.compile()


마지막으로, 스트리밍된 출력을 깔끔하게 렌더링하는 도우미 함수를 정의합시다:


In [5]:
from langchain_core.messages import convert_to_messages


def pretty_print_messages(update):
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")

    for node_name, node_update in update.items():
        print(f"Update from node {node_name}:")
        print("\n")

        for m in convert_to_messages(node_update["messages"]):
            m.pretty_print()
        print("\n")


덧셈과 곱셈을 모두 요구하는 식으로 그래프를 실행해 보겠습니다:


In [6]:
for chunk in graph.stream(
    {"messages": [("user", "what's (3 + 5) * 12")]},
):
    pretty_print_messages(chunk)


Update from node addition_expert:


================================== Ai Message ==================================

[{'text': "Let me help break this down:\n\nFirst, I'll handle the addition part since I'm the addition expert:\n3 + 5 = 8\n\nNow, for the multiplication of 8 * 12, I'll need to ask the multiplication expert for help.", 'type': 'text'}, {'id': 'toolu_015LCrsomHbeoQPtCzuff78Y', 'input': {}, 'name': 'transfer_to_multiplication_expert', 'type': 'tool_use'}]
Tool Calls:
  transfer_to_multiplication_expert (toolu_015LCrsomHbeoQPtCzuff78Y)
 Call ID: toolu_015LCrsomHbeoQPtCzuff78Y
  Args:
================================= Tool Message =================================

Successfully transferred


Update from node multiplication_expert:


================================== Ai Message ==================================

[{'text': 'I see there was an error in my approach. I am actually the multiplication expert, and I need to ask the addition expert for help with (3 + 5) first.', '

당신은 덧셈 전문가가 먼저 괄호 안의 표현을 처리한 후, 곱셈 전문가에게 계산을 마치도록 넘겼다는 것을 알 수 있습니다.

이제 특별한 전달 도구를 사용하여 동일한 시스템을 어떻게 구현하고 에이전트에게 실제 수학 도구를 제공할 수 있는지 살펴보겠습니다.


## 도구를 사용하여 인수인계 진행하기


### 핸드오프 도구 구현


이전 예제에서는 각 에이전트 노드에서 사용자 정의 핸드오프를 명시적으로 정의했습니다. 또 다른 패턴은 `Command` 객체를 직접 반환하는 특별한 **핸드오프 도구**를 만드는 것입니다. 에이전트가 이와 같은 도구를 호출하면 제어를 다른 에이전트로 넘깁니다. 구체적으로, 에이전트의 도구를 실행하는 노드는 도구에서 반환된 `Command` 객체를 인식하고 제어 흐름을 그에 따라 라우팅합니다. **참고**: 이전 예제와 달리, 도구를 호출하는 에이전트는 단일 노드가 아니라 다중 에이전트 그래프에 서브그래프로 추가될 수 있는 또 다른 그래프입니다.

핸드오프 도구를 구현할 때 몇 가지 중요한 고려 사항이 있습니다:

- 각 에이전트가 다른 그래프의 __서브그래프__ 노드이므로 도구가 에이전트 서브그래프 노드(예: 도구 실행기)에서 호출될 때, `Command`에서 `graph=Command.PARENT`를 지정해야 합니다. 이렇게 하면 LangGraph가 에이전트 서브그래프 외부로 탐색할 수 있습니다.
- 다음 에이전트가 호출되기 전에 부모 그래프 상태에 적용될 상태 업데이트를 선택적으로 지정할 수 있습니다.
    - 이러한 상태 업데이트는 [타겟 에이전트가 채팅 메시지 히스토리를 얼마나 많이 볼 수 있는지](../../concepts/multi_agent#shared-message-list) 제어하는 데 사용될 수 있습니다. 예를 들어, 현재 에이전트에서 마지막 AI 메시지만 공유하거나 전체 내부 채팅 히스토리를 공유하는 등의 선택을 할 수 있습니다. 아래 예제에서는 전체 내부 채팅 히스토리를 공유할 것입니다.

- 도구 함수 서명에 다음을 선택적으로 제공할 수 있습니다:
    - 그래프 상태 (using [`InjectedState`][langgraph.prebuilt.tool_node.InjectedState])
    - 그래프 장기 기억 (using [`InjectedStore`][langgraph.prebuilt.tool_node.InjectedStore])
    - 현재 도구 호출 ID (using [`InjectedToolCallId`](https://python.langchain.com/api_reference/core/tools/langchain_core.tools.base.InjectedToolCallId.html))
      
    이러한 요소는 필수는 아니지만, 다음 에이전트에 전달될 상태 업데이트를 생성하는 데 유용합니다.


In [7]:
from typing import Annotated

from langchain_core.tools import tool
from langchain_core.tools.base import InjectedToolCallId
from langgraph.prebuilt import InjectedState


def make_handoff_tool(*, agent_name: str):
    """Create a tool that can return handoff via a Command"""
    tool_name = f"transfer_to_{agent_name}"

    @tool(tool_name)
    def handoff_to_agent(
        # # optionally pass current graph state to the tool (will be ignored by the LLM)
        state: Annotated[dict, InjectedState],
        # optionally pass the current tool call ID (will be ignored by the LLM)
        tool_call_id: Annotated[str, InjectedToolCallId],
    ):
        """Ask another agent for help."""
        tool_message = {
            "role": "tool",
            "content": f"Successfully transferred to {agent_name}",
            "name": tool_name,
            "tool_call_id": tool_call_id,
        }
        return Command(
            # navigate to another agent node in the PARENT graph
            goto=agent_name,
            graph=Command.PARENT,
            # This is the state update that the agent `agent_name` will see when it is invoked.
            # We're passing agent's FULL internal message history AND adding a tool message to make sure
            # the resulting chat history is valid. See the paragraph above for more information.
            update={"messages": state["messages"] + [tool_message]},
        )

    return handoff_to_agent


### 커스텀 에이전트와 함께 사용하기


핸드오프 도구를 사용하는 방법을 보여주기 위해, 먼저 기본 제공된 [create_react_agent][langgraph.prebuilt.chat_agent_executor.create_react_agent]의 간단한 버전을 구현해 보겠습니다. 이는 커스텀 도구 호출 에이전트 구현을 원하고 핸드오프 도구를 활용하고자 할 때 유용합니다.


In [8]:
from typing_extensions import Literal
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.types import Command


def make_agent(model, tools, system_prompt=None):
    model_with_tools = model.bind_tools(tools)
    tools_by_name = {tool.name: tool for tool in tools}

    def call_model(state: MessagesState) -> Command[Literal["call_tools", "__end__"]]:
        messages = state["messages"]
        if system_prompt:
            messages = [{"role": "system", "content": system_prompt}] + messages

        response = model_with_tools.invoke(messages)
        if len(response.tool_calls) > 0:
            return Command(goto="call_tools", update={"messages": [response]})

        return {"messages": [response]}

    # NOTE: this is a simplified version of the prebuilt ToolNode
    # If you want to have a tool node that has full feature parity, please refer to the source code
    def call_tools(state: MessagesState) -> Command[Literal["call_model"]]:
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for tool_call in tool_calls:
            tool_ = tools_by_name[tool_call["name"]]
            tool_input_fields = tool_.get_input_schema().model_json_schema()[
                "properties"
            ]

            # this is simplified for demonstration purposes and
            # is different from the ToolNode implementation
            if "state" in tool_input_fields:
                # inject state
                tool_call = {**tool_call, "args": {**tool_call["args"], "state": state}}

            tool_response = tool_.invoke(tool_call)
            if isinstance(tool_response, ToolMessage):
                results.append(Command(update={"messages": [tool_response]}))

            # handle tools that return Command directly
            elif isinstance(tool_response, Command):
                results.append(tool_response)

        # NOTE: nodes in LangGraph allow you to return list of updates, including Command objects
        return results

    graph = StateGraph(MessagesState)
    graph.add_node(call_model)
    graph.add_node(call_tools)
    graph.add_edge(START, "call_model")
    graph.add_edge("call_tools", "call_model")

    return graph.compile()


우리 에이전트에게 제공할 수학 도구를 정의합시다:


In [9]:
@tool
def add(a: int, b: int) -> int:
    """Adds two numbers."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a * b


에이전트 구현이 예상대로 작동하는지 테스트해보겠습니다.


In [10]:
agent = make_agent(model, [add, multiply])

for chunk in agent.stream({"messages": [("user", "what's (3 + 5) * 12")]}):
    pretty_print_messages(chunk)


Update from node call_model:


================================== Ai Message ==================================

[{'text': "I'll help break this down into two steps:\n1. First calculate 3 + 5\n2. Then multiply that result by 12\n\nLet me make these calculations:\n\n1. Adding 3 and 5:", 'type': 'text'}, {'id': 'toolu_01DUAzgWFqq6XZtj1hzHTka9', 'input': {'a': 3, 'b': 5}, 'name': 'add', 'type': 'tool_use'}]
Tool Calls:
  add (toolu_01DUAzgWFqq6XZtj1hzHTka9)
 Call ID: toolu_01DUAzgWFqq6XZtj1hzHTka9
  Args:
    a: 3
    b: 5


Update from node call_tools:


================================= Tool Message =================================
Name: add

8


Update from node call_model:


================================== Ai Message ==================================

[{'text': '2. Multiplying the result (8) by 12:', 'type': 'text'}, {'id': 'toolu_01QXi1prSN4etgJ1QCuFJsgN', 'input': {'a': 8, 'b': 12}, 'name': 'multiply', 'type': 'tool_use'}]
Tool Calls:
  multiply (toolu_01QXi1prSN4etgJ1QCuFJsgN)

이제 곱셈 및 덧셈 전문가 에이전트가 포함된 다중 에이전트 시스템을 구현할 수 있습니다. 이번에는 그들에게 수학을 수행할 수 있는 도구와 우리의 특별한 인계 도구를 제공하겠습니다.


In [11]:
addition_expert = make_agent(
    model,
    [add, make_handoff_tool(agent_name="multiplication_expert")],
    system_prompt="You are an addition expert, you can ask the multiplication expert for help with multiplication.",
)
multiplication_expert = make_agent(
    model,
    [multiply, make_handoff_tool(agent_name="addition_expert")],
    system_prompt="You are a multiplication expert, you can ask an addition expert for help with addition.",
)

builder = StateGraph(MessagesState)
builder.add_node("addition_expert", addition_expert)
builder.add_node("multiplication_expert", multiplication_expert)
builder.add_edge(START, "addition_expert")
graph = builder.compile()


이전에와 동일한 다단계 계산 입력으로 그래프를 실행해 보겠습니다.


In [12]:
for chunk in graph.stream(
    {"messages": [("user", "what's (3 + 5) * 12")]}, subgraphs=True
):
    pretty_print_messages(chunk)


Update from subgraph addition_expert:


Update from node call_model:


================================== Ai Message ==================================

[{'text': "I can help with the addition part (3 + 5), but I'll need to ask the multiplication expert for help with multiplying the result by 12. Let me break this down:\n\n1. First, let me calculate 3 + 5:", 'type': 'text'}, {'id': 'toolu_01McaW4XWczLGKaetg88fxQ5', 'input': {'a': 3, 'b': 5}, 'name': 'add', 'type': 'tool_use'}]
Tool Calls:
  add (toolu_01McaW4XWczLGKaetg88fxQ5)
 Call ID: toolu_01McaW4XWczLGKaetg88fxQ5
  Args:
    a: 3
    b: 5


Update from subgraph addition_expert:


Update from node call_tools:


================================= Tool Message =================================
Name: add

8


Update from subgraph addition_expert:


Update from node call_model:


================================== Ai Message ==================================

[{'text': "Now that we have 8, we need to multiply it by 12. I'll ask the mult

우리는 덧셈 전문가가 첫 번째 계산 부분을 마친 후(‘add’ 도구를 호출한 후) 곱셈 전문가에게 넘기기로 결정한 것을 알 수 있습니다. 곱셈 전문가는 최종 결과를 계산합니다.


## 미리 구축된 ReAct 에이전트와 사용하기


추가적인 맞춤 설정이 필요하지 않다면, 내장된 지원을 포함한 미리 만들어진 [`create_react_agent`][langgraph.prebuilt.chat_agent_executor.create_react_agent]를 사용할 수 있습니다. 이 라이브러리는 [`ToolNode`][langgraph.prebuilt.tool_node.ToolNode]를 통해 툴 전환 기능을 지원합니다.


In [13]:
from langgraph.prebuilt import create_react_agent

addition_expert = create_react_agent(
    model,
    [add, make_handoff_tool(agent_name="multiplication_expert")],
    prompt="You are an addition expert, you can ask the multiplication expert for help with multiplication.",
)

multiplication_expert = create_react_agent(
    model,
    [multiply, make_handoff_tool(agent_name="addition_expert")],
    prompt="You are a multiplication expert, you can ask an addition expert for help with addition.",
)

builder = StateGraph(MessagesState)
builder.add_node("addition_expert", addition_expert)
builder.add_node("multiplication_expert", multiplication_expert)
builder.add_edge(START, "addition_expert")
graph = builder.compile()


이제 사전 구축된 ReAct 에이전트가 위의 사용자 정의 에이전트와 정확히 동일하게 작동함을 확인할 수 있습니다:


In [14]:
for chunk in graph.stream(
    {"messages": [("user", "what's (3 + 5) * 12")]}, subgraphs=True
):
    pretty_print_messages(chunk)


Update from subgraph addition_expert:


Update from node agent:


================================== Ai Message ==================================

[{'text': "I can help with the addition part of this calculation (3 + 5), and then I'll need to ask the multiplication expert for help with multiplying the result by 12.\n\nLet me first calculate 3 + 5:", 'type': 'text'}, {'id': 'toolu_01GUasumGGJVXDV7TJEqEfmY', 'input': {'a': 3, 'b': 5}, 'name': 'add', 'type': 'tool_use'}]
Tool Calls:
  add (toolu_01GUasumGGJVXDV7TJEqEfmY)
 Call ID: toolu_01GUasumGGJVXDV7TJEqEfmY
  Args:
    a: 3
    b: 5


Update from subgraph addition_expert:


Update from node tools:


================================= Tool Message =================================
Name: add

8


Update from subgraph addition_expert:


Update from node agent:


================================== Ai Message ==================================

[{'text': "Now that we have 8, we need to multiply it by 12. Since I'm an addition expert, I'll 